# Regression problem

In [1]:
import pandas as pd 
import numpy as np
from kiwigrad import MLP, Value, Layer
from sklearn.metrics import r2_score
import time

### Dataset

In [2]:
X = pd.read_csv("data/x.csv", sep=";")
print(X.shape)
X.head()

(414, 6)


,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude
0,-0.823683,1.255628,-0.792495,2.007407,1.125430,0.448762
1,-0.823683,0.157086,-0.616612,1.667503,0.912444,0.401139
2,1.541151,-0.387791,-0.414015,0.307885,1.486860,0.688183
3,1.246435,-0.387791,-0.414015,0.307885,1.486860,0.688183
4,-1.121951,-1.117223,-0.549997,0.307885,0.834188,0.592937


In [3]:
Xn = X.to_numpy().tolist()
inputs = [[Value(i) for i in elem] for elem in Xn]
inputs[0]

[Value(data=-0.823683, grad=0.000000),
 Value(data=1.255628, grad=0.000000),
 Value(data=-0.792495, grad=0.000000),
 Value(data=2.007407, grad=0.000000),
 Value(data=1.125430, grad=0.000000),
 Value(data=0.448762, grad=0.000000)]

In [4]:
y = pd.read_csv("data/y.csv", sep=";")
y = y.to_numpy().squeeze()
print(y.shape)

(414,)


### Multi layer perceptron

In [5]:
class PotNet(MLP):
    def __init__(self):
        layers = [
            Layer(nin=2, nout=16, bias=True, activation="relu"),
            Layer(nin=16, nout=16, bias=True, activation="relu"),
            Layer(nin=16, nout=1, bias=True, activation="linear")
        ]
        super().__init__(layers=layers)

model = PotNet()
y_pred = model(inputs[0])
print(y_pred)

Value(data=-1.170386, grad=0.000000)


### Full battch gradient descent with MSE loss function

In [6]:
epochs = 200
lr = 0.001
for epoch in range(epochs):
    s = time.monotonic()
    for i in range(len(inputs)):
        output = model(inputs[i])
        target = Value(y[i])
        loss = ((output-target) ** 2)
        loss.backward()
        for p in model.parameters():
            p.data += -lr * p.grad
        model.zero_grad()
    e = time.monotonic()
    t = e - s
    if epoch%10:
        print(f"epoch: {epoch} | loss: {loss.data:.2f} | time: {t:.2f} sec.")

epoch: 1 | loss: 688.63 | time: 0.14 sec.
epoch: 2 | loss: 676.58 | time: 0.23 sec.
epoch: 3 | loss: 667.44 | time: 0.14 sec.
epoch: 4 | loss: 678.55 | time: 0.13 sec.
epoch: 5 | loss: 674.23 | time: 0.13 sec.
epoch: 6 | loss: 659.47 | time: 0.13 sec.
epoch: 7 | loss: 665.66 | time: 0.13 sec.


: 

: 